# Proposition 1: February Discount for Lucky Customer

This SQL script creates two tables, `dbo.Orders` and `dbo.OrderLinesWithDiscount`, with primary and foreign key constraints and specific data validations. It then inserts data from existing tables `Sales.Orders` and `Sales.OrderLines` into these newly created tables. The script filters and updates data for a specific customer, applying a discount to order lines meeting specific conditions.

### Objectives:

1. **Create New Tables**: Define and create the `dbo.Orders` and `dbo.OrderLinesWithDiscount` tables with necessary constraints.
2. **Insert Data**: Populate the new tables with data from `Sales.Orders` and `Sales.OrderLines`.
3. **Filter Data by Customer and Date**: Retrieve records for a specific customer (CustomerID 183) with orders within a specified date range.
4. **Update Discount**: Apply a discount to the order lines for the selected customer within the specified date range.

## Functional Specification

### Inputs

- **Sales.Orders**: Contains customer orders with fields `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate`.
- **Sales.OrderLines**: Contains order line items, including `OrderLineID`, `OrderID`, `StockItemID`, `UnitPrice`, and `Quantity`.

### Outputs

- **dbo.Orders**: A new table with orders for each customer, with constraints on `OrderID` (primary key) and `CustomerID`.
- **dbo.OrderLinesWithDiscount**: A new table with order line items, including a `Discount` field and several constraints on `UnitPrice`, `Quantity`, and `Discount`.
- **Filtered Orders and Order Lines**: Selected rows filtered by `CustomerID` and `OrderDate`.
- **Updated Discount**: Order line items for `CustomerID` 183 within the specified date range, updated with a 50% discount.

### Detailed Breakdown

1. **Table Creation**
    
    - **dbo.Orders**:
        - Creates the table with columns `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate`.
        - `OrderID` is the primary key.
    - **dbo.OrderLinesWithDiscount**:
        - Creates the table with columns `OrderLineID`, `OrderID`, `StockItemID`, `UnitPrice`, `Quantity`, and `Discount`.
        - Constraints:
            - `OrderLineID`: Primary key.
            - `OrderID`: Foreign key referencing `dbo.Orders(OrderID)`.
            - `UnitPrice`: Default value is 0, and it must be a non-negative number.
            - `Quantity`: Default value is 1, and it must be a positive integer.
            - `Discount`: Default value is 0, and it must be between 0 and 1.
2. **Data Insertion**
    
    - **Insert Data into dbo.Orders**:
        - Populates `dbo.Orders` with records from `Sales.Orders`.
    - **Insert Data into dbo.OrderLinesWithDiscount**:
        - Populates `dbo.OrderLinesWithDiscount` with records from `Sales.OrderLines`.
3. **Filtering Orders and Order Lines by Customer and Date**
    
    - Selects all records from `dbo.OrderLinesWithDiscount` for a specific customer (`CustomerID = 183`) and date range (`OrderDate` between '2016-02-01' and '2016-02-29').
4. **Applying a Discount to Order Lines**
    
    - Updates the `Discount` field to 0.50 for all `OrderLinesWithDiscount` records that meet the criteria (CustomerID 183 and order date range '2016-02-01' to '2016-02-29').
5. **Verification of Updated Orders and Order Lines**
    
- Retrieves updated records from `dbo.Orders` and `dbo.OrderLinesWithDiscount` for `CustomerID = 183` to verify the discount application.

### Edge Cases

- **Discount Constraint Violation**: If the `Discount` field is updated with a value outside the range \[0, 1\], the constraint `CHK_Discount` will prevent the update.
- **No Orders in Date Range**: If `CustomerID = 183` has no orders between '2016-02-01' and '2016-02-29', no rows will be updated or selected in the filtered query.

### Assumptions

- `Sales.Orders` and `Sales.OrderLines` tables exist and contain accurate data needed for initial insertions.
- `CustomerID 183` is a valid customer with orders within the specified date range.

In [ ]:
DROP TABLE IF EXISTS dbo.OrderLinesWithDiscount, dbo.Orders;

CREATE TABLE dbo.Orders
(
    OrderID INT NOT NULL
        CONSTRAINT PK_Orders PRIMARY KEY(OrderID),
    CustomerID INT NOT NULL,
    OrderDate DATE NOT NULL,
    ExpectedDeliveryDate DATE NOT NULL
);

CREATE TABLE dbo.OrderLinesWithDiscount
(
    OrderLineID INT NOT NULL
        CONSTRAINT PK_OrderLines PRIMARY KEY(OrderLineID),
    OrderID INT NOT NULL
        CONSTRAINT FK_OrderLines_Order FOREIGN KEY(OrderID)
        REFERENCES dbo.Orders(OrderID),
    StockItemID INT NOT NULL,
    UnitPrice DECIMAL(18,2) NULL
        CONSTRAINT DFT_OrderLines_UnitPrice DEFAULT(0),
    CONSTRAINT CHK_UnitPrice CHECK (UnitPrice >= 0),
    Quantity INT NOT NULL
        CONSTRAINT DFT_OrderLines_Quantity DEFAULT(1),
    CONSTRAINT CHK_Quantity CHECK (Quantity > 0),
    Discount NUMERIC(4, 3) NOT NULL
        CONSTRAINT DFT_OrderLines_Discount DEFAULT(0),
    CONSTRAINT CHK_Discount CHECK (Discount BETWEEN 0 AND 1)
);
GO

INSERT INTO dbo.Orders
    (OrderID, CustomerID, OrderDate, ExpectedDeliveryDate)
SELECT OrderID, CustomerID, OrderDate, ExpectedDeliveryDate
FROM Sales.Orders;
INSERT INTO dbo.OrderLinesWithDiscount
    (OrderLineID, OrderID, StockItemID, UnitPrice, Quantity)
SELECT OrderLineID, OrderID, StockItemID, UnitPrice, Quantity
FROM Sales.OrderLines;


In [ ]:
SELECT *
FROM dbo.OrderLinesWithDiscount
WHERE OrderID IN (SELECT OrderID
    FROM dbo.Orders
    WHERE CustomerID = 183 AND OrderDate BETWEEN '2016-02-01' AND '2016-02-29')
ORDER BY OrderID;

UPDATE OL
SET Discount = 0.50
FROM dbo.OrderLinesWithDiscount AS OL
    INNER JOIN dbo.Orders AS O
    ON OL.OrderID = O.OrderID
WHERE O.CustomerID = 183 AND O.OrderDate BETWEEN '2016-02-01' AND '2016-02-29';

SELECT *
FROM dbo.Orders
WHERE CustomerID = 183 AND OrderDate BETWEEN '2016-02-01' AND '2016-02-29';

SELECT *
FROM dbo.OrderLinesWithDiscount
WHERE OrderID IN (SELECT OrderID
    FROM dbo.Orders
    WHERE CustomerID = 183 AND OrderDate BETWEEN '2016-02-01' AND '2016-02-29')
ORDER BY OrderID;

# Proposition 2: Inserting Customer 381

This SQL script inserts a new order into the `dbo.Orders` table for a customer with `CustomerID` 381. It sets the `OrderID` for the new order by incrementing the maximum existing `OrderID` by one. The script then retrieves all orders associated with `CustomerID` 381, including the newly inserted order, to confirm the insertion.

### Key Objectives:

1. **Generate a New Order ID**: Determine the maximum `OrderID` in the `dbo.Orders` table and set the `OrderID` for the new record as `MAX(OrderID) + 1`.
2. **Insert the New Order**: Add a new row to the `dbo.Orders` table with the incremented `OrderID`, specified `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate`.
3. **Retrieve Customer Orders**: Display all orders associated with `CustomerID` 381 to confirm the new order’s successful insertion.

## Functional Specification

### Inputs

- **dbo.Orders table**: Table containing order records with fields for `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate`.
- **Parameter values**:
    - `CustomerID = 381`: Customer ID for whom the new order is created.
    - `OrderDate = '2024-10-28'`: Date on which the order was placed.
    - `ExpectedDeliveryDate = '2024-10-28'`: Expected delivery date for the order.

### Outputs

- **dbo.Orders table**: A new order record will be added for `CustomerID` 381.
- **Confirmation**: A result set displaying all orders for `CustomerID` 381, including the newly inserted order, for verification.

### Script Breakdown

1. **Declare Last Order ID Variable**:
    
    - Declares an integer variable `@lastOrderID` to store the maximum existing `OrderID` in the `dbo.Orders` table.
2. **Determine New Order ID**:
    
    - Sets `@lastOrderID` to the maximum current `OrderID` in the `dbo.Orders` table. This step identifies the last used `OrderID` so the new `OrderID` can be incremented by one.
3. **Insert New Order**:
    
    - Adds a new order for `CustomerID` 381 into the `dbo.Orders` table using the `@lastOrderID + 1` as the `OrderID`, with the specified `OrderDate` and `ExpectedDeliveryDate`.
4. **Confirm Insertion**:
    
- Selects all orders associated with `CustomerID` 381 to verify the insertion of the new order.

### Edge Cases

- **Empty `dbo.Orders` Table**: If the `dbo.Orders` table is empty, `MAX(OrderID)` will return `NULL`, potentially causing the script to fail. Adding a conditional check for `NULL` and initializing `@lastOrderID` to 0 if the table is empty would prevent this issue.
- **Order Date Validation**: The script assumes the provided `OrderDate` and `ExpectedDeliveryDate` are valid and in the correct format.

### Assumptions

- The `OrderID` is numeric and increments sequentially with each new order.
- `dbo.Orders` table is indexed on `CustomerID` to optimize retrieval operations for customer-specific orders.

In [ ]:
DECLARE @lastOrderID INT;
SET @lastOrderID = (SELECT MAX(OrderID) FROM dbo.Orders);

INSERT INTO dbo.Orders
    (OrderID, CustomerID, OrderDate, ExpectedDeliveryDate)
VALUES (@lastOrderID +1, 381, '2024-10-28', '2024-10-28');

SELECT * FROM dbo.Orders WHERE CustomerID = 381;

# Proposition 3: The Purge of a Hundred and One

The SQL script performs the following actions:

1. **Delete an Existing Table (if it exists)**: Drops the table `OrderLinesWithDiscount` if it already exists in the database.
2. **Delete Rows from Orders Table**: Deletes all records from the `Orders` table where the `CustomerID` falls within a specified range (between 100 and 200).
3. **Retrieve Orders**: Selects and displays all records from the `Orders` table where `CustomerID` is within a slightly expanded range (from 99 to 201) for verification, ordering the results by `CustomerID`.

### Key Objectives

- Ensure that the `OrderLinesWithDiscount` table does not exist, likely in preparation for creating or rebuilding it.
- Remove specific rows from the `Orders` table based on `CustomerID` range.
- Retrieve and verify records in a defined range to confirm deletion was successful.

* * *

## Functional Specification

### Inputs

- **dbo.OrderLinesWithDiscount Table**: A table potentially storing order line information with applied discounts.
- **dbo.Orders Table**: A table containing orders, with fields including `CustomerID` and other order-related data.

### Outputs

- **Order Deletion**: All records in `dbo.Orders` where `CustomerID` is between 100 and 200 are permanently removed.
- **Selected Order Records**: After deletion, the query returns records where `CustomerID` is between 99 and 201, displaying remaining records within this range.

### Query Breakdown

1. **Drop Table (if Exists)**:
    
    - Drops the `OrderLinesWithDiscount` table if it already exists in the database. This step is likely performed as part of a larger data preparation or reinitialization process.
2. **Delete Orders within Specified Range**:
    
    - Deletes rows in `dbo.Orders` where `CustomerID` is between 100 and 200. This could be part of a data cleanup, targeting specific customers for removal based on `CustomerID`.
3. **Select and Verify Orders**:
    
- Retrieves all records in `dbo.Orders` where `CustomerID` is between 99 and 201, ordered by `CustomerID`. This final step allows verification of the delete operation by displaying remaining orders in the affected range.

### Edge Cases

- **Non-Existent Table**: If `OrderLinesWithDiscount` doesn’t exist, `DROP TABLE IF EXISTS` prevents errors by checking for existence before dropping.
- **Customer Range without Orders**: If there are no orders with `CustomerID` between 100 and 200, no rows are deleted.
- **Expanded Range Verification**: The final select query shows records within an expanded range (99 to 201) to verify deletion boundaries.

### Assumptions

- The `CustomerID` range (100-200) is deliberately chosen to target specific customer records for deletion.
- The `dbo.Orders` table uses `CustomerID` as an identifier and is appropriately indexed for efficient querying.

In [ ]:
DROP TABLE IF EXISTS dbo.OrderLinesWithDiscount;

DELETE FROM dbo.Orders
WHERE CustomerID BETWEEN 100 AND 200;

SELECT *
FROM dbo.Orders
WHERE CustomerID BETWEEN 99 AND 201
ORDER BY CustomerID;

# Proposition 4: Renovations for Salesperson 7

This SQL code defines a stored procedure named `Sales.GetOrdersBySalesperson` that retrieves orders based on a specific salesperson's ID. It then clears the `dbo.Orders` table, populates it with orders for a salesperson with the ID of 7, and finally retrieves and displays the records from both `dbo.Orders` and `Sales.Orders` tables in ascending `OrderID` order.

### Objectives:

1. **Create/Modify a Stored Procedure**: Define or alter a stored procedure to retrieve orders by a specified salesperson.
2. **Data Manipulation**: Clear the contents of the `dbo.Orders` table and insert orders retrieved by the stored procedure.
3. **Data Retrieval and Display**: Display all records in `dbo.Orders` and filter `Sales.Orders` based on the specified salesperson.

* * *

## Functional Specification

### Inputs

- **@SalespersonPersonID**: An integer parameter that specifies the salesperson whose orders are to be retrieved.

### Outputs

- **OrderID**: Unique identifier for each order.
- **CustomerID**: Identifier for the customer placing the order.
- **OrderDate**: Date the order was placed.
- **ExpectedDeliveryDate**: Date the order is expected to be delivered.

### Process Overview

1. **Stored Procedure Creation**:
    
    - `CREATE OR ALTER PROC Sales.GetOrdersBySalesperson` defines a stored procedure that accepts `@SalespersonPersonID` as a parameter.
    - The procedure selects `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate` from the `Sales.Orders` table, filtering results by `SalespersonPersonID`.
2. **Truncation of `dbo.Orders` Table**:
    
    - `TRUNCATE TABLE dbo.Orders` clears all data in the `dbo.Orders` table without logging individual row deletions, preparing it for fresh data insertion.
3. **Data Insertion**:
    
    - The `INSERT INTO dbo.Orders` statement uses `EXEC Sales.GetOrdersBySalesperson @SalespersonPersonID = 7` to insert data into the `dbo.Orders` table by invoking the stored procedure with a specific salesperson ID (7).
    - This populates `dbo.Orders` with order data for the specified salesperson.
4. **Data Retrieval**:
    
- `SELECT * FROM dbo.Orders ORDER BY OrderID` retrieves all data from `dbo.Orders` ordered by `OrderID`.
- The second `SELECT` statement retrieves orders from `Sales.Orders` for `SalespersonPersonID = 7`, also ordered by `OrderID`.

### Edge Cases

- **Invalid SalespersonPersonID**: If an invalid `@SalespersonPersonID` is provided, the procedure will return no records, and no data will be inserted into `dbo.Orders`.
- **Data Truncation Impact**: The `TRUNCATE` operation fully clears the `dbo.Orders` table, so any pre-existing data will be lost. Ensure no essential data is in `dbo.Orders` before execution.

### Assumptions

- The `Sales.Orders` and `dbo.Orders` tables share similar schemas, allowing direct insertion from `Sales.GetOrdersBySalesperson`.
- The stored procedure `Sales.GetOrdersBySalesperson` has appropriate permissions to access and return data from the `Sales.Orders` table.

In [ ]:
CREATE OR ALTER PROC Sales.GetOrdersBySalesperson
    @SalespersonPersonID AS INT
AS
    SELECT OrderID, CustomerID, OrderDate, ExpectedDeliveryDate
    FROM Sales.Orders
    WHERE SalespersonPersonID = @SalespersonPersonID;
GO

TRUNCATE TABLE dbo.Orders;

INSERT INTO dbo.Orders(OrderID, CustomerID, OrderDate, ExpectedDeliveryDate)
    EXEC Sales.GetOrdersBySalesperson @SalespersonPersonID = 7;

SELECT * 
FROM dbo.Orders 
ORDER BY OrderID;

SELECT OrderID, CustomerID, OrderDate, ExpectedDeliveryDate
FROM Sales.Orders
WHERE SalespersonPersonID = 7
ORDER BY OrderID;

# Proposition 5: Instant Table Duplication (but without some orderlines)

The SQL script creates a new table, `dbo.OrderLines`, based on data from the `Sales.OrderLines` table, excluding records with `OrderLineID` values between 10 and 30. The script removes any existing `dbo.OrderLines` table to ensure there are no conflicts. Once the new table is created, it selects all records from `dbo.OrderLines` and orders them by `OrderID` and `OrderLineID`.

### Key Objectives:

1. **Create `dbo.OrderLines` Table**: Populate a new table in the database (`dbo.OrderLines`) with records from `Sales.OrderLines`, excluding those with `OrderLineID` values between 10 and 30.
2. **Ensure Table Uniqueness**: Drop the existing `dbo.OrderLines` table, if it exists, before creating a new one to avoid conflicts or data duplication.
3. **Display Results**: Query all rows from the new `dbo.OrderLines` table, ordered by `OrderID` and `OrderLineID`.

## Functional Specification

### Inputs

- **Sales.OrderLines Table**: This source table contains the original records, including `OrderLineID`, `OrderID`, `StockItemID`, `Quantity`, and `UnitPrice`.

### Outputs

- **dbo.OrderLines Table**: A new table containing all records from `Sales.OrderLines` except for those where `OrderLineID` is between 10 and 30.
- **Result Set**: A query output displaying all data from `dbo.OrderLines`, sorted by `OrderID` and `OrderLineID`.

### Query Breakdown

1. **Remove Existing Table**:
    
    - `DROP TABLE IF EXISTS dbo.OrderLines;` checks if `dbo.OrderLines` already exists and drops it to avoid conflicts during the table creation process.
2. **Create New `dbo.OrderLines` Table**:
    
    - The `INTO dbo.OrderLines` clause creates a new table based on the result set from the `EXCEPT` operation.
    - The `EXCEPT` operation copies all records from `Sales.OrderLines` except those with `OrderLineID` between 10 and 30.
3. **Display Ordered Data**:
    
- The final `SELECT *` query retrieves all data from the new `dbo.OrderLines` table and orders the result by `OrderID` and `OrderLineID` for better readability and data organization.

### Edge Cases

- **No Rows Match `EXCEPT` Condition**: If there are no rows with `OrderLineID` between 10 and 30 in `Sales.OrderLines`, the entire dataset from `Sales.OrderLines` is copied to `dbo.OrderLines`.
- **Duplicate Rows**: If any rows in `Sales.OrderLines` are duplicated (except for `OrderLineID`), they will all appear in `dbo.OrderLines` except those specifically excluded.

### Assumptions

- The `Sales.OrderLines` table is available and accessible with the expected columns (`OrderLineID`, `OrderID`, `StockItemID`, `Quantity`, `UnitPrice`).
- The new table `dbo.OrderLines` will not be impacted by constraints or triggers, as these are not mentioned.

In [ ]:
DROP TABLE IF EXISTS dbo.OrderLines;

SELECT OrderLineID, OrderID, StockItemID, Quantity, UnitPrice
INTO dbo.OrderLines
FROM Sales.OrderLines

EXCEPT

SELECT OrderLineID, OrderID, StockItemID, Quantity, UnitPrice
FROM Sales.OrderLines
WHERE OrderLineID BETWEEN 10 AND 30;

SELECT *
FROM dbo.OrderLines
ORDER BY OrderID, OrderLineID;

# Proposition 6: Merging Tables

This SQL script copies a subset of data from the `Sales.Orders` table into a temporary table called `dbo.OrdersTwo`, using `OrderID` values between 1 and 50. The script then synchronizes data between `dbo.OrdersTwo` and the main `dbo.Orders` table using the `MERGE` statement, ensuring that records with matching `OrderID`s are updated and those without matches are inserted. The script concludes by displaying the updated contents of the `dbo.Orders` table.

### Key Objectives:

1. **Create a Temporary Table**: Generate a subset of orders in a temporary table `dbo.OrdersTwo` for use in the data merge.
2. **Synchronize Tables Using `MERGE`**: Update existing records and insert new ones from `dbo.OrdersTwo` into `dbo.Orders`.
3. **Display Final Data**: Show all records in `dbo.Orders` post-merge.

## Functional Specification

### Inputs

- **Sales.Orders table**: Contains the original set of order records, including `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate`.

### Outputs

- **dbo.OrdersTwo table**: A subset of orders with `OrderID` values from 1 to 50 from the `Sales.Orders` table.
- **dbo.Orders table (updated)**: The `dbo.Orders` table, updated based on the `MERGE` operation with `dbo.OrdersTwo`.

### Query Breakdown

1. **Temporary Table Creation**:
    
    - The script drops `dbo.OrdersTwo` if it exists to avoid conflicts.
    - It then selects `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate` from `Sales.Orders` for orders with `OrderID` between 1 and 50, and inserts these into a new table `dbo.OrdersTwo`.
2. **Display Initial Data**:
    
    - A `SELECT` query retrieves and displays all rows from `dbo.Orders`, ordered by `OrderID`, to show the initial state before the merge.
3. **MERGE Operation**:
    
    - The `MERGE` statement synchronizes data between `dbo.OrdersTwo` and `dbo.Orders`.
    - **When records match (`OrderID` exists in both tables)**:
        - The `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate` columns in `dbo.Orders` are updated to reflect the values from `dbo.OrdersTwo`.
    - **When records do not match (`OrderID` is missing in `dbo.Orders`)**:
        - The record is inserted into `dbo.Orders` with the values from `dbo.OrdersTwo`.
4. **Display Final Data**:
    
- After the `MERGE` operation, a final `SELECT` statement retrieves all rows from `dbo.Orders`, ordered by `OrderID`, to show the table’s updated state.

### Edge Cases

- **Duplicate OrderIDs in `Sales.Orders`**: If `Sales.Orders` has multiple records with the same `OrderID` in the 1–50 range, the `SELECT INTO` query may encounter errors due to duplicate primary keys.
- **Missing Fields**: If `Sales.Orders` has null values for critical fields, this could impact the quality of data transferred to `dbo.OrdersTwo` and subsequently to `dbo.Orders`.

### Assumptions

- The `Sales.Orders` and `dbo.Orders` tables have appropriate primary key constraints, ensuring unique `OrderID`s in each table.
- Any necessary permissions for creating, dropping, and merging tables are in place.

In [ ]:
DROP TABLE IF EXISTS dbo.OrdersTwo;

SELECT OrderID, CustomerID, OrderDate, ExpectedDeliveryDate
INTO dbo.OrdersTwo
FROM Sales.Orders
WHERE OrderID BETWEEN 1 AND 50;

SELECT *
FROM dbo.Orders
ORDER BY OrderID;

MERGE INTO dbo.Orders AS TGT
USING dbo.OrdersTwo AS SRC
ON TGT.OrderID = SRC.OrderID
WHEN MATCHED THEN
    UPDATE SET
    TGT.CustomerID = SRC.CustomerID,
    TGT.OrderDate = SRC.OrderDate,
    TGT.ExpectedDeliveryDate = SRC.ExpectedDeliveryDate
WHEN NOT MATCHED THEN
    INSERT (OrderID, CustomerID, OrderDate, ExpectedDeliveryDate)
    VALUES (SRC.OrderID, SRC.CustomerID, SRC.OrderDate, SRC.ExpectedDeliveryDate);
    
SELECT *
FROM dbo.Orders
ORDER BY OrderID;

# Proposition 7: Making an Employees Table

The SQL script is used to manage the `dbo.Employees` table, which stores a list of employees by creating a new table structure and inserting data from an existing source. The script performs the following steps:

1. **Table Management**: It first checks if the `dbo.Employees` table already exists and drops it if so.
2. **Table Creation**: The script then creates a new `dbo.Employees` table with columns for `EmployeeID` and `FullName`.
3. **Data Insertion**: Finally, it inserts employee data into `dbo.Employees` from the `Application.People` table, where individuals marked as employees are included.

### Key Objectives

- **Ensure Data Consistency**: Guarantee that the `dbo.Employees` table reflects the current list of employees without duplicating or retaining obsolete data.
- **Efficiently Insert Data**: Use the `OUTPUT` clause to provide immediate feedback on inserted data for tracking or logging purposes.

## Functional Specification

### Inputs

- **Application.People table**: Contains a list of individuals with details including their `FullName` and `IsEmployee` status.

### Outputs

- **dbo.Employees table**: Stores employee data, including:
    - `EmployeeID`: Unique identifier for each employee, generated as an `IDENTITY`.
    - `FullName`: Name of the employee, copied from the `Application.People` table.
- **Inserted Data Feedback**: The script outputs the `EmployeeID` and `FullName` for each newly inserted employee record.

### Script Breakdown

1. **Table Deletion**:
    
    - `DROP TABLE IF EXISTS dbo.Employees;` is used to delete the `dbo.Employees` table if it already exists to prevent errors during table creation and ensure no residual data is present.
2. **Table Creation**:
    
    - `CREATE TABLE dbo.Employees` creates a new table structure with two columns:
        - `EmployeeID`: Defined as an integer primary key with an `IDENTITY` constraint for auto-incremented unique values.
        - `FullName`: A string with a maximum length of 50 characters, representing the name of the employee.
    - `PK_EmployeeID` primary key constraint is applied to ensure that `EmployeeID` values are unique.
3. **Data Insertion**:
    
- The `INSERT INTO dbo.Employees` statement inserts records from `Application.People`, where `IsEmployee = 1` indicates that the individual is an employee.
- The `OUTPUT` clause returns each `EmployeeID` and `FullName` from the inserted records for immediate visibility and can be used for confirmation, logging, or further processing.

### Edge Cases

- **No Employees in Application.People**: If there are no records where `IsEmployee = 1`, no data will be inserted into `dbo.Employees`.
- **Duplicate Employee Names**: The script does not prevent duplicate `FullName` values, assuming it’s allowed for multiple employees to share names.

### Assumptions

- The `Application.People` table is assumed to contain accurate employee data, including a correctly maintained `IsEmployee` flag.
- The `FullName` field length is appropriate, and 50 characters is sufficient for all employee names.

In [ ]:
DROP TABLE IF EXISTS dbo.Employees;

CREATE TABLE dbo.Employees
(
    EmployeeID INT NOT NULL IDENTITY(1, 1) CONSTRAINT PK_EmployeeID PRIMARY KEY,
    FullName NVARCHAR(50) NOT NULL
);

INSERT INTO dbo.Employees(FullName)
    OUTPUT inserted.EmployeeID, inserted.FullName
        SELECT FullName
        FROM Application.People
        WHERE IsEmployee = 1;

# Proposition 8: Pricing Problem

 <span style="color: var(--vscode-foreground);">This SQL query updates the </span> `UnitPrice` <span style="color: var(--vscode-foreground);"> for each order line of a specific customer (CustomerID = 15) by setting it equal to the </span> `TotalPrice` <span style="color: var(--vscode-foreground);"> (calculated as </span> `UnitPrice * Quantity`<span style="color: var(--vscode-foreground);">). The query uses a Common Table Expression (CTE) named </span> `T` <span style="color: var(--vscode-foreground);"> to first select order lines for the given customer and calculate the </span> `TotalPrice`<span style="color: var(--vscode-foreground);">. The </span> `OUTPUT` <span style="color: var(--vscode-foreground);"> clause then provides details on the updated values for each order line, showing both old and new values for </span> `UnitPrice` <span style="color: var(--vscode-foreground);"> and </span> `TotalPrice`<span style="color: var(--vscode-foreground);">.</span>

### Key Objectives

1. **Select and Calculate Total Price**: Retrieve order lines for `CustomerID = 15`, then calculate the `TotalPrice` as `UnitPrice * Quantity`.
2. **Update Unit Price**: Update `UnitPrice` to match `TotalPrice` for these order lines.
3. **Return Results**: Use the `OUTPUT` clause to return the old and new values of `UnitPrice` and `TotalPrice`.

## Functional Specification

### Inputs

- **dbo.OrderLines table**: Contains individual items or lines within an order, including fields `UnitPrice`, `OrderID`, and `Quantity`.
- **dbo.Orders table**: Contains orders with details such as `OrderID` and `CustomerID`.

### Outputs

- **OldUnitPrice**: The original `UnitPrice` before the update.
- **NewUnitPrice**: The updated `UnitPrice`, which is set to `TotalPrice`.
- **TotalPrice**: The original calculated `TotalPrice` (before the update).
- **NewTotalPrice**: The `TotalPrice` after updating `UnitPrice`, which remains the same since it’s based on the `UnitPrice` value at the time of retrieval.

### Query Breakdown

1. **Common Table Expression (CTE) - `T`**:
    
    - This CTE retrieves order lines for `CustomerID = 15` by joining the `dbo.OrderLines` and `dbo.Orders` tables on `OrderID`.
    - For each retrieved order line, it calculates `TotalPrice` as `UnitPrice * Quantity`.
2. **Update Operation**:
    
    - The main query updates the `UnitPrice` in CTE `T` to equal the `TotalPrice` calculated in the CTE.
3. **Output Clause**:
    
- The `OUTPUT` clause is used to provide details of each update, including:
    - `OldUnitPrice`: The original `UnitPrice` before the update.
    - `NewUnitPrice`: The updated value of `UnitPrice`.
    - `TotalPrice`: The original `TotalPrice`.
    - `NewTotalPrice`: The `TotalPrice` after the update (since `TotalPrice` in the CTE `T` remains constant, it is equivalent to the old `TotalPrice`).

### Edge Cases

- **No Matching CustomerID**: If there are no order lines for `CustomerID = 15`, the query performs no updates, and the `OUTPUT` result is empty.
- **Zero or Null Quantities**: If any `Quantity` values are zero or null, `TotalPrice` will be zero or null respectively, which could result in a zero or null `UnitPrice`.

### Assumptions

- It is intended that `UnitPrice` be set equal to `TotalPrice` and not calculated dynamically (i.e., `TotalPrice` is fixed as `UnitPrice * Quantity` at the time of selection in the CTE `T`).
- The `dbo.OrderLines` table is updatable through a CTE.

In [ ]:
WITH T AS
(
SELECT UnitPrice, UnitPrice * Quantity AS TotalPrice
FROM dbo.OrderLines AS OD
    INNER JOIN dbo.Orders AS O ON OD.OrderID = O.OrderID
WHERE O.CustomerID = 15
)
UPDATE T
    SET UnitPrice = TotalPrice
OUTPUT
    deleted.UnitPrice AS OldUnitPrice,
    inserted.UnitPrice AS NewUnitPrice,
    deleted.TotalPrice AS TotalPrice,
    inserted.TotalPrice AS NewTotalPrice;

# Proposition 9: Clearing History

The SQL `DELETE` statement removes records from the `dbo.Orders` table where the `OrderDate` is on or before June 30, 2015. For each deleted row, it returns specific details (`OrderID`, `CustomerID`, `OrderDate`, `ExpectedDeliveryDate`) using the `OUTPUT` clause to help log or track the deleted orders.

### Key Objectives:

1. **Remove Old Orders**: Deletes orders that were placed on or before the cutoff date of June 30, 2015.
2. **Track Deleted Records**: Outputs the details of each deleted order for tracking and validation.

## Functional Specification

### Inputs

- **dbo.Orders table**: Contains order records, including `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate`.

### Outputs

- **OrderID**: Unique identifier of the deleted order.
- **CustomerID**: ID of the customer associated with the deleted order.
- **OrderDate**: Date when the deleted order was originally placed.
- **ExpectedDeliveryDate**: Expected delivery date for the deleted order.

### Query Breakdown

1. **Row Deletion**:
    - The `DELETE` statement targets rows in the `dbo.Orders` table where `OrderDate` is less than or equal to June 30, 2015.
2. **Output Deleted Information**:
- The `OUTPUT` clause captures information from each row that is deleted, specifically the `OrderID`, `CustomerID`, `OrderDate`, and `ExpectedDeliveryDate`.
- This information can be used for record-keeping, auditing, or transferring data before permanent deletion.

### Edge Cases

- **No Matching Records**: If there are no orders with `OrderDate` on or before June 30, 2015, the query will perform no deletions, and the output will be empty.
- **Null Values in ExpectedDeliveryDate**: If any deleted orders have a `NULL` value in `ExpectedDeliveryDate`, it will be reflected as `NULL` in the output.

### Assumptions

- The `dbo.Orders` table is properly backed up before performing this deletion.
- There is a business need to remove records on or before June 30, 2015, possibly as part of data archiving or storage management.

In [ ]:
DELETE FROM dbo.Orders
    OUTPUT
        deleted.OrderID,
        deleted.CustomerID,
        deleted.OrderDate,
        deleted.ExpectedDeliveryDate
WHERE OrderDate <= '2015-06-30';

# Proposition 10: Happy Names!
This SQL update query modifies the `FullName` field in the `dbo.Employees` table by appending an exclamation mark (`!`) to the names of employees whose names start with one of the letters A, B, C, D, or E. The query outputs the original and updated names for each affected row.
### Key Objectives:
1. **Update Employee Names**: Append an exclamation mark to names that begin with specific letters.
2. **Output Changes**: Return a result set that includes the old and new values of `FullName` for each row that was updated.
* * *
## Functional Specification
### Inputs
- **dbo.Employees table**: Contains employee records, including a `FullName` column with employee names.
### Outputs
- **OldName**: The previous value of the `FullName` field before the update.
- **NewName**: The updated value of the `FullName` field after appending the exclamation mark.
### Query Breakdown
1. **Condition for Update**:
    
    - The `WHERE` clause limits updates to only those rows where `FullName` starts with one of the letters A, B, C, D, or E. This is achieved using the `LIKE N'[ABCDE]%'` pattern.
2. **Update Operation**:
    
    - The `SET` clause appends an exclamation mark to `FullName` for each row that meets the condition.
3. **Output Clause**:
    
- The `OUTPUT` clause is used to return both the old (`deleted.FullName`) and new (`inserted.FullName`) values of `FullName` for each row that was modified.
### Edge Cases
- **Names Already Ending with '!'**: If any names already end with `!`, this will add an additional `!`.
- **No Matches**: If no `FullName` values begin with A, B, C, D, or E, no rows will be updated, and the output will be empty.
### Assumptions
- The `dbo.Employees` table is available and contains a `FullName` column with valid Unicode strings.
- Business logic permits appending `!` to employee names as an indicator or for stylistic purposes.

In [7]:
UPDATE dbo.Employees
    SET FullName = FullName + N'!'
OUTPUT
    deleted.FullName AS OldName,
    inserted.FullName AS NewName
WHERE FullName LIKE N'[ABCDE]%';